[![Open in Colab](hhtps://colab.research.google.com/assets/cloab-badge.svg)](https://colab.research.google.com/drive/1DvqoamIBAQ9weAOnIW7ST8e0NHPNam_x)

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torch.nn as nn
!pip install torchdiffeq
import torchdiffeq
!pip install torchtext==0.6.0

from models import ODEBlock
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.16.0
    Uninstalling torchtext-0.16.0:
      Successfully uninstalled torchtext-0.16.0


In [2]:
print(device)

cpu


### Construction du modèle

In [47]:
import math
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len):
        super().__init__()

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        self.pe = torch.zeros(max_len, 1, d_model).to(device)
        self.pe[:, 0, 0::2] = torch.sin(position * div_term)
        self.pe[:, 0, 1::2] = torch.cos(position * div_term)

    def forward(self, x):
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return x

In [46]:
class EncoderMHAttentionODEFunc(nn.Module):
    """MLP modeling the derivative of FeedForward ODE system.
    device : torch.device
    data_dim : int
        Dimension of data.
    hidden_dim : int
        Dimension of hidden layers.
    augment_dim: int
        Dimension of augmentation. If 0 does not augment ODE, otherwise augments
        it with augment_dim dimensions.
    """
    def __init__(self, device, data_dim,  augment_dim=0, n_heads=1, time_dependent = False):
        super(EncoderMHAttentionODEFunc, self).__init__()
        self.device = device
        self.augment_dim = augment_dim
        self.data_dim = data_dim
        self.input_dim = data_dim + augment_dim + time_dependent
        self.nfe = 0  # Number of function evaluations
        self.time_dependent = time_dependent

        self.n_heads = n_heads
        self.att = nn.MultiheadAttention(self.input_dim, self.n_heads)

    def forward(self, t, x):
        """
        t : torch.Tensor (not used here)
            Current time. Shape (1,).
        x : torch.Tensor
            Shape (batch_size, input_dim)
        """
        # Forward pass of model corresponds to one function evaluation, so
        # increment counter
        if self.time_dependent:
          batch_size, len_seq, emb_size = x.shape
          t_ = t*torch.ones(batch_size, len_seq,1).to(device)
          x_ = torch.cat([x,t_], 2)
        else :
          x_=x
        self.nfe += 1
        x_ = self.att(x_, x_, x_)[0][:,:,:self.input_dim-self.time_dependent]
        return x

class FeedForwardODEFunc(nn.Module):
    """MLP modeling the derivative of FeedForward ODE system.
    device : torch.device
    data_dim : int
        Dimension of data.
    hidden_dim : int
        Dimension of hidden layers.
    augment_dim: int
        Dimension of augmentation. If 0 does not augment ODE, otherwise augments
        it with augment_dim dimensions.
    """
    def __init__(self, device, data_dim, hidden_dim, augment_dim=0, time_dependent = False):
        super(FeedForwardODEFunc, self).__init__()
        self.device = device
        self.augment_dim = augment_dim
        self.data_dim = data_dim
        self.time_dependent = time_dependent
        self.input_dim = data_dim + augment_dim + time_dependent
        self.hidden_dim = hidden_dim
        self.nfe = 0  # Number of function evaluations
        self.layers = nn.Sequential(
            nn.Linear(self.input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, self.input_dim - time_dependent),
        )

    def forward(self, t, x):
        """
        t : torch.Tensor (not used here)
            Current time. Shape (1,).
        x : torch.Tensor
            Shape (batch_size, input_dim)
        """
        # Forward pass of model corresponds to one function evaluation, so
        # increment counter
        if self.time_dependent:
          batch_size, len_seq, emb_size = x.shape
          t_ = t*torch.ones(batch_size, len_seq,1).to(device)
          x_ = torch.cat([x,t_], 2)
        else :
          x_=x
        self.nfe += 1
        return self.layers(x_)

In [45]:
class EncoderBlockFunc(nn.Module):
  def __init__(self, device, data_dim, hidden_dim, augment_dim=0, n_heads = 1, time_dependent = False):
        super(EncoderBlockFunc, self).__init__()
        self.device = device
        self.augment_dim = augment_dim
        self.data_dim = data_dim
        self.input_dim = data_dim + augment_dim
        self.hidden_dim = hidden_dim
        self.nfe = 0  # Number of function evaluations
        self.n_heads = n_heads
        self.mha = EncoderMHAttentionODEFunc(device, data_dim, augment_dim = augment_dim, n_heads = n_heads, time_dependent = time_dependent)
        self.ffd = FeedForwardODEFunc(device, data_dim, hidden_dim, augment_dim=augment_dim, time_dependent = time_dependent)
        self.layn1 = nn.LayerNorm(self.input_dim)
        self.layn2 = nn.LayerNorm(self.input_dim)

  def forward(self, t, x):
        """
        t : torch.Tensor (not used here)
            Current time. Shape (1,).
        x : torch.Tensor
            Shape (batch_size, input_dim)
        """
        # Forward pass of model corresponds to one function evaluation, so
        # increment counter
        self.nfe += 1
        x = self.mha(t,x) + x
        x = self.layn1(x)
        x = self.ffd(t,x) + x
        x = self.layn2(x)
        return x

class SumEncoderBlockFunc(nn.Module):
  def __init__(self, device, data_dim, hidden_dim, augment_dim=0, n_heads = 1, time_dependent = False):
        super(SumEncoderBlockFunc, self).__init__()
        self.device = device
        self.augment_dim = augment_dim
        self.data_dim = data_dim
        self.input_dim = data_dim + augment_dim
        self.hidden_dim = hidden_dim
        self.nfe = 0  # Number of function evaluations
        self.n_heads = n_heads
        self.time_dependent = time_dependent
        self.mha = EncoderMHAttentionODEFunc(device, data_dim, augment_dim = augment_dim, n_heads = n_heads, time_dependent = time_dependent)
        self.ffd = FeedForwardODEFunc(device, data_dim, hidden_dim, augment_dim=augment_dim, time_dependent = time_dependent)

  def forward(self, t, x):
        """
        t : torch.Tensor (not used here)
            Current time. Shape (1,).
        x : torch.Tensor
            Shape (batch_size, input_dim)
        """
        # Forward pass of model corresponds to one function evaluation, so
        # increment counter
        self.nfe += 1
        return (self.mha(t,x) + self.ffd(t,x))/2

In [44]:
class ODETransformerClassification(nn.Module):
  def __init__(self, device, seq_len, emb_size,  hidden_dim, n_blocks = 1,  augment_dim=0, n_heads = 1, time_dependent = False):
    super(ODETransformerClassification, self).__init__()
    self.time_dependent = time_dependent
    self.device = device
    self.seq_len = seq_len
    self.emb_size = emb_size
    self.hidden_dim = hidden_dim
    self.augment_dim = augment_dim
    self.n_heads = n_heads
    ode_blocks = []
    funcs = []
    func_names = []
    for i in range(n_blocks):
      encoder_func = EncoderMHAttentionODEFunc(device, emb_size+ 2*i*augment_dim , augment_dim=augment_dim, n_heads=n_heads, time_dependent = time_dependent)
      ode_blocks.append(ODEBlock(device, encoder_func, is_seq=True))
      funcs.append(encoder_func)
      func_names.append(f"MHA {i+1}")
      ode_blocks.append(nn.LayerNorm(emb_size+ (2*i+1)*augment_dim))
      ff_func = FeedForwardODEFunc(device, emb_size+ (2*i+1)*augment_dim, hidden_dim, augment_dim=augment_dim, time_dependent = time_dependent)
      ode_blocks.append(ODEBlock(device, ff_func, is_seq=True))
      ode_blocks.append(nn.LayerNorm(emb_size+ (2*i+2)*augment_dim))
      funcs.append(ff_func)
      func_names.append(f"FFD {i+1}")
    self.ode_blocks = ode_blocks

    self.funcs = funcs
    self.func_names = func_names

    self.block_layers = nn.Sequential(*ode_blocks)
    self.final_layer = nn.Sequential(nn.Linear((emb_size+2*augment_dim*n_blocks)*seq_len, 1), nn.Sigmoid())
    self.pos_encoding = PositionalEncoding(emb_size, seq_len)

  def forward(self, x):
    x=x.to(self.device)
    x = self.pos_encoding(x)
    x = self.block_layers(x).view(x.shape[0], -1)
    return self.final_layer(x)

  def get_NFEs(self):
    return {self.func_names[i]: self.funcs[i].nfe for i in range(len(self.funcs))}


class ODETransformerClassificationAllinOne(nn.Module):
  def __init__(self, device, seq_len, emb_size,  hidden_dim, n_blocks = 1,  augment_dim=0, n_heads = 1, time_dependent = False):
    super(ODETransformerClassificationAllinOne, self).__init__()
    self.device = device
    self.seq_len = seq_len
    self.emb_size = emb_size
    self.hidden_dim = hidden_dim
    self.augment_dim = augment_dim
    self.n_heads = n_heads
    ode_blocks = []
    funcs = []
    func_names = []
    for i in range(n_blocks):
      encoder_block = EncoderBlockFunc(device,emb_size+i*augment_dim, hidden_dim,augment_dim=augment_dim, n_heads = n_heads, time_dependent = time_dependent)
      ode_blocks.append(ODEBlock(device, encoder_block, is_seq=True))
      funcs.append(encoder_block)
      func_names.append(f"Encoder {i+1}")
    self.funcs = funcs
    self.func_names = func_names
    self.block_layers = nn.Sequential(*ode_blocks)
    self.final_layer = nn.Sequential(nn.Linear((emb_size+augment_dim*n_blocks)*seq_len, 1), nn.Sigmoid())
    self.pos_encoding = PositionalEncoding(emb_size, seq_len)

  def forward(self, x):
    x = self.pos_encoding(x)
    x = self.block_layers(x).view(x.shape[0], -1)
    return self.final_layer(x)

  def get_NFEs(self):
    return {self.func_names[i]: self.funcs[i].nfe for i in range(len(self.funcs))}

class ODETransformerClassificationAllinOneSum(nn.Module):
  def __init__(self, device, seq_len, emb_size,  hidden_dim, n_blocks = 1,  augment_dim=0, n_heads = 1, time_dependent = False):
    super(ODETransformerClassificationAllinOneSum, self).__init__()
    self.device = device
    self.seq_len = seq_len
    self.emb_size = emb_size
    self.hidden_dim = hidden_dim
    self.time_dependent = time_dependent
    self.augment_dim = augment_dim
    self.n_heads = n_heads
    ode_blocks = []
    funcs = []
    func_names = []
    for i in range(n_blocks):
      encoder_block = SumEncoderBlockFunc(device,emb_size+i*augment_dim, hidden_dim,augment_dim=augment_dim, n_heads = n_heads, time_dependent = time_dependent)
      ode_blocks.append(ODEBlock(device, encoder_block, is_seq=True))
      ode_blocks.append(nn.LayerNorm(emb_size + augment_dim*(i+1)))
      funcs.append(encoder_block)
      func_names.append(f"Additive Encoder {i+1}")

    self.block_layers = nn.Sequential(*ode_blocks)
    self.final_layer = nn.Sequential(nn.Linear((emb_size+augment_dim*n_blocks)*seq_len, 1), nn.Sigmoid())
    self.pos_encoding = PositionalEncoding(emb_size, seq_len)

  def forward(self, x):
    x = self.pos_encoding(x)
    x = self.block_layers(x).view(x.shape[0], -1)
    return self.final_layer(x)
  def get_NFEs(self):
    return {self.func_names[i]: self.funcs[i].nfe for i in range(len(self.funcs))}

### Importation des données

In [7]:
import torchtext as tt

emb_dim = 50
# Load pre-trained GloVe embeddings
glove = tt.vocab.GloVe(name='6B', dim=emb_dim)

# Define tokenizer
tokenizer = tt.data.utils.get_tokenizer("basic_english")

# Define fields for text and label
TEXT = tt.data.Field(lower=True, include_lengths=True, batch_first=True, tokenize=tokenizer)
LABEL = tt.data.Field(sequential=False)

# Define batch size, maximum review length, and maximum vocabulary words
batch_size = 64
max_review_len = 100
max_vocab_words = 3500

# Load IMDb dataset
train_ds, test_ds = tt.datasets.IMDB.splits(TEXT, LABEL)

# Build vocabulary
TEXT.build_vocab(train_ds, max_size=max_vocab_words-2)
LABEL.build_vocab(train_ds)

# Split train dataset into train and dev sets
train_ds, dev_ds = train_ds.split(split_ratio=0.8)

# Create data iterators
train_loader, dev_loader, test_loader = tt.data.BucketIterator.splits(
    (train_ds, dev_ds, test_ds),
    batch_sizes=(batch_size, batch_size, batch_size),
    shuffle=True,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
)

.vector_cache/glove.6B.zip: 862MB [02:39, 5.40MB/s]                           
100%|█████████▉| 399999/400000 [00:14<00:00, 27514.44it/s]


downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:08<00:00, 9.55MB/s]


In [8]:
max_length = 128
# Convert tokenized text into embedded tensors
def get_embedded_text(loader):
    embedded_texts = []
    targets = []
    for batch in loader:
        text, lengths = batch.text
        target = batch.label.unsqueeze(1)
        target = torch.where(target == 1, torch.tensor(0.), torch.tensor(1.))  # Add dimension for concatenation


        embedded_text = []
        for sentence, length in zip(text, lengths):
            sentence_emb = []
            for i in range(min(length.item(), max_length)):
                word = TEXT.vocab.itos[sentence[i]]
                try:
                    word_emb = glove.vectors[glove.stoi[word]]
                except KeyError:
                    word_emb = glove.vectors[glove.stoi['unk']]
                sentence_emb.append(word_emb)
            # Padding
            sentence_emb += [torch.zeros(glove.vectors.shape[1])] * (max_length - length)
            embedded_text.append(torch.stack(sentence_emb))

        embedded_texts.append(torch.stack(embedded_text))
        targets.append(target)

    embedded_texts = torch.cat(embedded_texts)
    targets = torch.cat(targets).squeeze(1)
    return embedded_texts, targets


# Get embedded tensors for train, dev, and test datasets
train_embedded = get_embedded_text(train_loader)
print("Train embedded text tensor shape:", train_embedded[0].shape)


dev_embedded = get_embedded_text(dev_loader)
print("Dev embedded text tensor shape:", dev_embedded[0].shape)

test_embedded = get_embedded_text(test_loader)
print("Test embedded text tensor shape:", test_embedded[0].shape)

Train embedded text tensor shape: torch.Size([20000, 128, 50])
Dev embedded text tensor shape: torch.Size([5000, 128, 50])
Test embedded text tensor shape: torch.Size([25000, 128, 50])


In [9]:
from torch.utils.data import Dataset, DataLoader
class CustomDataset(Dataset):
    def __init__(self, data):
        self.text = data[0]
        self.target = data[1]

    def __len__(self):
        return self.text.size(0)

    def __getitem__(self, idx):
        return self.text[idx], self.target[idx]


trainset_embedded = CustomDataset(train_embedded)
devset_embedded = CustomDataset(dev_embedded)
testset_embedded = CustomDataset(test_embedded)

traindataloader_embedded = DataLoader(trainset_embedded, batch_size=32, shuffle=True)
testdataloader_embedded = DataLoader(testset_embedded, batch_size=64, shuffle=True)
devdataloader_embedded = DataLoader(devset_embedded, batch_size=64, shuffle=True)

### Entrainement et test

In [40]:
model = ODETransformerClassification(device, max_length, emb_dim, 32, augment_dim=0, time_dependent= True)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCELoss()
print(sum(p.numel() for p in model.parameters()))
NFEs = {cle:[] for cle in model.func_names}


20523


In [48]:
from tqdm import tqdm

def train(dataloader, devdataloader, model, optimizer, criterion, update_NFEs = False, with_test = False):
    model.train()
    total_loss = 0.0
    total_samples = 0
    model.to(device)
    new_NFEs = {cle:[] for cle in model.func_names}
    for batch in tqdm(dataloader):
        inputs, targets = batch
        inputs = inputs.to(device)  # Move inputs to GPU if available
        targets = targets.to(device)  # Move targets to GPU if available
        optimizer.zero_grad()  # Clear gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs.view(-1), targets.view(-1))  # Calculate loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights
        total_loss += loss.item() * len(inputs)
        total_samples += len(inputs)
        if update_NFEs:
          batch_NFEs = model.get_NFEs()
          for cle in model.func_names:
            new_NFEs[cle].append(batch_NFEs[cle])
    if update_NFEs:
      for cle in model.func_names:
        NFEs[cle].append(sum(new_NFEs[cle])/len(new_NFEs[cle]))
    if with_test:
      with torch.no_grad():
        model.eval()
        correct = 0
        total_loss_dev = 0
        total_dev = 0
        for inputs, targets in tqdm(devdataloader):
          outputs = model(inputs.to(device))
          total_loss_dev += criterion(outputs.view(-1), targets.view(-1))*len(inputs)
          preds = torch.where(outputs.view(-1) <0.5, torch.tensor(0.), torch.tensor(1.))
          correct += torch.sum(preds==targets.view(-1)).item()
          total_dev += len(inputs)
        return total_loss / total_samples, total_loss_dev/total_dev, correct/total_dev

    return total_loss/total_samples



for i in range(15):
  loss, dev_loss, dev_accuracy = train(traindataloader_embedded, devdataloader_embedded, model, optimizer, criterion, with_test=True, update_NFEs =True)
  print(f"Epoch {i+1} : Loss = {loss}")

  2%|▏         | 12/625 [00:07<06:18,  1.62it/s]


KeyboardInterrupt: 

In [ ]:
with torch.no_grad():
  model.eval()
  correct = 0
  total_loss = 0
  total = 0
  for inputs, targets in tqdm(testdataloader_embedded):
    outputs = model(inputs.to(device))
    total_loss += criterion(outputs.view(-1), targets.view(-1))*len(inputs)
    preds = torch.where(outputs.view(-1) <0.5, torch.tensor(0.), torch.tensor(1.))
    correct += torch.sum(preds==targets.view(-1)).item()
    total += len(inputs)

print(f"Test loss : {total_loss/total}")
print(f"Test accuracy : {correct/total}")